In [8]:
import numpy as np
import gym
import tensorflow as tf
#from tensorflow.keras.layers import Dense, Flatten
from keras.layers import Dense, Flatten
#from tensorflow.keras.models import Sequential
from keras.models import Sequential
#from tensorflow.keras.optimizers import RMSprop
from keras import optimizers

from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy,LinearAnnealedPolicy, BoltzmannQPolicy, GreedyQPolicy, MaxBoltzmannQPolicy, BoltzmannGumbelQPolicy, SoftmaxPolicy
from rl.core import Processor

In [9]:
def create_agent(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(48, activation='relu'))
    model.add(Dense(48, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [10]:
class TaxiProcessor(Processor):
    def process_observation(self, observation):
        one_hot = np.zeros(500)
        one_hot[observation] = 1
        return one_hot

    def process_reward(self, reward):
        return reward

In [11]:
env = gym.make("Taxi-v3")
print("No of state = ", env.observation_space.n)
print("No of actions =", env.action_space.n)
model = create_agent(500, env.action_space.n)
model.summary()

No of state =  500
No of actions = 6
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 500)               0         
                                                                 
 dense_5 (Dense)             (None, 24)                12024     
                                                                 
 dense_6 (Dense)             (None, 48)                1200      
                                                                 
 dense_7 (Dense)             (None, 48)                2352      
                                                                 
 dense_8 (Dense)             (None, 24)                1176      
                                                                 
 dense_9 (Dense)             (None, 6)                 150       
                                                                 
Total params: 16,

In [12]:
memory = SequentialMemory(limit=800000, window_length=1)
processor = TaxiProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.05, value_test=.005, nb_steps=10000)

#policy = LinearAnnealedPolicy(MaxBoltzmannQPolicy(), attr='eps', value_max=1., value_min=.05, value_test=.005, nb_steps=10000)
dqn = DQNAgent(model=model, nb_actions=env.action_space.n, processor=processor, memory=memory, nb_steps_warmup=1000, gamma=0.99, policy=policy, enable_double_dqn=True, target_model_update=5e-3)



In [13]:
optimizer = tf.keras.optimizers.legacy.RMSprop(lr=1e-3)
optimizer1 = optimizers.Adam(lr=1e-3)
dqn.compile(optimizer1, metrics=['mae'])
dqn.fit(env, nb_steps=250000, visualize=False, verbose=1)

Training for 250000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 71s 7ms/step - reward: -2.5843
51 episodes - episode_reward: -503.804 [-803.000, -15.000] - loss: 0.772 - mae: 13.752 - mean_q: -12.985 - mean_eps: 0.478 - prob: 1.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 80s 8ms/step - reward: -1.1329
53 episodes - episode_reward: -212.642 [-263.000, 9.000] - loss: 2.184 - mae: 25.523 - mean_q: -26.340 - mean_eps: 0.050 - prob: 1.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: -1.1953
50 episodes - episode_reward: -239.960 [-668.000, -200.000] - loss: 3.714 - mae: 25.582 - mean_q: -22.653 - mean_eps: 0.050 - prob: 1.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 85s 9ms/step - reward: -1.1053
55 episodes - episode_reward: -203.691 [-254.000, -1.000] - loss: 1.700 - mae: 17.218 - mean_q: -15.646 - mea

In [22]:
env.reset()
res = dqn.test(env, nb_episodes=100, visualize=False)


Testing for 100 episodes ...
Episode 1: reward: 9.000, steps: 12
Episode 2: reward: 11.000, steps: 10
Episode 3: reward: 5.000, steps: 16
Episode 4: reward: 9.000, steps: 12
Episode 5: reward: 5.000, steps: 16
Episode 6: reward: 10.000, steps: 11
Episode 7: reward: 6.000, steps: 15
Episode 8: reward: 8.000, steps: 13
Episode 9: reward: 10.000, steps: 11
Episode 10: reward: 5.000, steps: 16
Episode 11: reward: 11.000, steps: 10
Episode 12: reward: 9.000, steps: 12
Episode 13: reward: 6.000, steps: 15
Episode 14: reward: 4.000, steps: 17
Episode 15: reward: 4.000, steps: 17
Episode 16: reward: 6.000, steps: 15
Episode 17: reward: 11.000, steps: 10
Episode 18: reward: 8.000, steps: 13
Episode 19: reward: 8.000, steps: 13
Episode 20: reward: 8.000, steps: 13
Episode 21: reward: 4.000, steps: 17
Episode 22: reward: 7.000, steps: 14
Episode 23: reward: 7.000, steps: 14
Episode 24: reward: 14.000, steps: 7
Episode 25: reward: 5.000, steps: 16
Episode 26: reward: 15.000, steps: 6
Episode 27: r

In [23]:
res.history
print(np.average(res.history['episode_reward']))

8.46
